In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Load the CSV file
metadata = pd.read_csv('ATEPP-metadata-1.1.csv')

# Filter data for compositions by Ludwig van Beethoven
beethoven_data = metadata[metadata['composer'] == 'Ludwig van Beethoven']

# Filter out necessary columns
filtered_metadata = beethoven_data[['artist', 'midi_path']]

# Get artist counts
artist_counts = filtered_metadata['artist'].value_counts()

# Identify the top 5 artists with the most records
top_artists = artist_counts.head(5).index

# Randomly select 300 records for each of the top artists
selected_data = filtered_metadata[filtered_metadata['artist'].isin(top_artists)]
sampled_data = selected_data.groupby('artist').apply(lambda x: x.sample(n=300, random_state=36)).reset_index(drop=True)

# Encode the 'artist' column
le = LabelEncoder()
sampled_data['artist_encoded'] = le.fit_transform(sampled_data['artist'])

# Split the data into training and testing sets
train_df, test_df = train_test_split(sampled_data, test_size=0.2, stratify=sampled_data['artist_encoded'], random_state=42)

# Reset the indices
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [ ]:
from tqdm.notebook import tqdm
import pretty_midi
import numpy as np

def process_midi_files_to_piano_rolls(midi_paths, fs=100, max_len=10000, pitch_range=(20, 100)):
    """
    Convert a list of MIDI files to padded piano roll representations with the same shape.
    
    Args:
        midi_paths (list): List of paths to MIDI files.
        fs (int): Sampling frequency for the piano roll.
        max_len (int, optional): The maximum length to pad or truncate sequences to. 
                                 If not provided, it uses the length of the longest sequence.
        pitch_range (tuple, optional): Tuple representing the range of pitches to retain. Defaults to (20, 100).
    
    Returns:
        numpy.ndarray: Array of padded piano roll representations with the same shape.
    """
    
    def midi_to_piano_roll(midi_path):
        try:
            midi_data = pretty_midi.PrettyMIDI(midi_path)
            piano_roll = midi_data.get_piano_roll(fs=fs)
            
            # Cut to the specified pitch range and transpose to have time on the x-axis
            return piano_roll[pitch_range[0]:pitch_range[1]].T
        except Exception as e:
            print(f"Error encountered while parsing file: {midi_path}, Error: {e}")
            return None
        
    padded_sequences = []
    for path in tqdm(midi_paths, desc="Processing MIDI files"):
        piano_roll = midi_to_piano_roll(path)
        if piano_roll is not None:
            if len(piano_roll) < max_len:
                num_padding = max_len - len(piano_roll)
                padded_seq = np.pad(piano_roll, ((0, num_padding), (0, 0)), 'constant')
            else:
                padded_seq = piano_roll[:max_len]
            padded_sequences.append(padded_seq)
    
    return np.array(padded_sequences)

In [ ]:
train_midi_paths = train_df['midi_path'].tolist()
test_midi_paths = test_df['midi_path'].tolist()

In [ ]:
train_padded_piano_rolls = process_midi_files_to_piano_rolls(train_midi_paths)
test_padded_piano_rolls = process_midi_files_to_piano_rolls(test_midi_paths)

In [ ]:
# Save the arrays
np.save('X_train_padded2.npy', train_padded_piano_rolls)
np.save('X_test_padded2.npy', test_padded_piano_rolls)

In [ ]:
import mido
from mido import MidiFile
from tqdm.notebook import tqdm
import numpy as np

def extract_features_from_midi(midi_file_path):
    # Load the MIDI file
    mid = MidiFile(midi_file_path)

    # Initialize lists to store IOIs, OTDs, and DLs
    iois = []
    ot_ds = []
    dl_s = []
    pedal_presses = []
    pedal_durations = []

    # Variables for polyphony and articulation
    notes_on = 0
    max_polyphony = 0
    legato_count = 0
    staccato_count = 0

    # Iterate through MIDI tracks and messages
    for track in mid.tracks:
        prev_time = 0
        prev_note_off_time = 0
        pedal_down_time = None
        for msg in track:
            # Calculate absolute time of the message
            absolute_time = prev_time + msg.time
            if msg.type == 'note_on' and msg.velocity > 0:
                notes_on += 1
                max_polyphony = max(max_polyphony, notes_on)

                # Calculate IOI (Inter-Onset Interval)
                ioi = absolute_time - prev_note_off_time
                iois.append(ioi)

                # Store the dynamic level (velocity)
                dl_s.append(msg.velocity)

                prev_time = absolute_time

            elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
                notes_on -= 1

                # Calculate OTD (Off-Time Duration)
                otd = absolute_time - prev_time
                ot_ds.append(otd)

                # Determine if the note is legato or staccato
                if otd >= ioi:
                    legato_count += 1
                else:
                    staccato_count += 1

                prev_note_off_time = absolute_time

            # Pedal usage
            if msg.type == 'control_change' and msg.control == 64:
                if msg.value > 0:  # Pedal pressed
                    pedal_presses.append(absolute_time)
                    pedal_down_time = absolute_time
                else:  # Pedal released
                    if pedal_down_time:
                        pedal_duration = absolute_time - pedal_down_time
                        pedal_durations.append(pedal_duration)
                        pedal_down_time = None

    # Calculate the standard deviations for IOI, OTD, and DL
    std_ioi = np.std(iois) if iois else 0
    std_otd = np.std(ot_ds) if ot_ds else 0
    std_dl = np.std(dl_s) if dl_s else 0

    # Calculate pedal usage metrics
    avg_pedal_duration = np.mean(pedal_durations) if pedal_durations else 0
    pedal_frequency = len(pedal_presses) / (mid.length if mid.length > 0 else 1)

    # Calculate legato to staccato ratio
    legato_staccato_ratio = legato_count / staccato_count if staccato_count > 0 else legato_count

    return np.array([std_ioi, std_otd, std_dl, max_polyphony, avg_pedal_duration, pedal_frequency, legato_staccato_ratio])

In [ ]:
train_features = [extract_features_from_midi(path) for path in tqdm(train_midi_paths, desc="Extracting Features")]
test_features = [extract_features_from_midi(path) for path in tqdm(test_midi_paths, desc="Extracting Features")]

In [ ]:
# Save the arrays
np.save('X_train_features4.npy', train_features)
np.save('X_test_features4.npy', test_features)